In [2]:
!pip install quantstats

     ---------------------------------------- 0.0/314.6 kB ? eta -:--:--
     -------------- ----------------------- 122.9/314.6 kB 2.4 MB/s eta 0:00:01
     -------------------------------------- 314.6/314.6 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     ------ --------------------------------- 0.5/3.0 MB 10.2 MB/s eta 0:00:01
     ----------- ---------------------------- 0.9/3.0 MB 11.0 MB/s eta 0:00:01
     ---------------- ----------------------- 1.2/3.0 MB 13.2 MB/s eta 0:00:01
     ----------------- 

In [13]:
import os
import pandas as pd
import quantstats as qs
# Path to the folder containing CSV files
folder_path = "Downloads/Broad Indices-20240330T071125Z-001/Broad Indices"

# Get a list of all CSV files in the folder
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]
dfs = []
# Iterate through each CSV file
for file in csv_files:
    filename = os.path.splitext(file)[0]
    # Construct the full path to the CSV file
    file_path = os.path.join(folder_path, file)
    
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)
    df['asset_name'] = filename
    
    data = metrics(df)
    dfs.append(data)
    
    
    
    #print(f"Processing file: {file}")
    #print(data.head())
    #print("\n")

combined_df = pd.concat(dfs, ignore_index=True)
print(combined_df)

            Date       Open       High        Low      Close  Adj Close  \
0     2007-05-30  91.580002  91.599998  91.500000  91.599998  78.043671   
1     2007-05-31  91.580002  91.599998  91.580002  91.599998  78.043671   
2     2007-06-01  91.620003  91.620003  91.620003  91.620003  78.060715   
3     2007-06-04  91.639999  91.639999  91.639999  91.639999  78.077782   
4     2007-06-05  91.660004  91.680000  91.639999  91.639999  78.077782   
...          ...        ...        ...        ...        ...        ...   
66601 2024-03-22  85.690002  85.879997  84.510002  84.570000  84.570000   
66602 2024-03-25  84.650002  84.959999  84.050003  84.050003  84.050003   
66603 2024-03-26  84.269997  84.370003  83.690002  83.709999  83.709999   
66604 2024-03-27  84.669998  85.900002  84.470001  85.870003  85.870003   
66605 2024-03-28  86.029999  86.769997  86.029999  86.480003  86.480003   

        Volume asset_name  ratio_sharpe  ratio_sortino  ratio_win_loss  \
0         1550        BIL

In [11]:
def metrics(data):
    """
    Calculate the Sharpe ratio for each asset based on their respective rows in the input DataFrame.

    Parameters:
    data (DataFrame): Input DataFrame containing historical price data with 'asset_name' as one of the columns.

    Returns:
    DataFrame: DataFrame with the Sharpe ratio calculated for each asset.
    """
    # Calculate the Sharpe ratio for each asset
    grouped_data = data.groupby("asset_name")


    data["ratio_sharpe"] = data["asset_name"].map(grouped_data.apply(lambda x: qs.stats.sharpe(x["Close"])))
    data["ratio_sortino"] = data["asset_name"].map(grouped_data.apply(lambda x: qs.stats.sortino(x["Close"])))
    data["ratio_win_loss"] = data["asset_name"].map(grouped_data.apply(lambda x: qs.stats.win_loss_ratio(x["Close"])))
    data["ratio_drawdown"] = data["asset_name"].map(grouped_data.apply(lambda x: qs.stats.max_drawdown(x["Close"])))

    # Calculate monthly and yearly CAGR for each asset
    data["Date"] = pd.to_datetime(data["Date"])
    data_1_d = data[data["Date"] >= data["Date"].max() - pd.DateOffset(days=1)]
    data_1_m = data[data["Date"] >= data["Date"].max() - pd.DateOffset(months=1)]
    data_3_m = data[data["Date"] >= data["Date"].max() - pd.DateOffset(months=3)]
    data_1_y = data[data["Date"] >= data["Date"].max() - pd.DateOffset(years=1)]

    data_1_d.set_index("Date", inplace=True)
    data_1_m.set_index("Date", inplace=True)
    data_3_m.set_index("Date", inplace=True)
    data_1_y.set_index("Date", inplace=True)

    data_1_d_grouped = data_1_d.groupby("asset_name")
    data_1_m_grouped = data_1_m.groupby("asset_name")
    data_3_m_grouped = data_3_m.groupby("asset_name")
    data_1_y_grouped = data_1_y.groupby("asset_name")


    # Assign CAGR values to the dataframe
    data["percentage_1_d_cagr"] = data["asset_name"].map(data_1_d_grouped.apply(lambda x: qs.stats.cagr(x["Close"])))
    data["percentage_1_m_cagr"] = data["asset_name"].map(data_1_m_grouped.apply(lambda x: qs.stats.cagr(x["Close"])))
    data["percentage_3_m_cagr"] = data["asset_name"].map(data_3_m_grouped.apply(lambda x: qs.stats.cagr(x["Close"])))
    data["percentage_1_y_cagr"] = data["asset_name"].map(data_1_y_grouped.apply(lambda x: qs.stats.cagr(x["Close"])))
    
    data["percentage_1_d_volatility"] = data["asset_name"].map(data_1_d_grouped.apply(lambda x: qs.stats.volatility(x["Close"])))
    data["percentage_1_m_volatility"] = data["asset_name"].map(data_1_m_grouped.apply(lambda x: qs.stats.volatility(x["Close"])))
    data["percentage_3_m_volatility"] = data["asset_name"].map(data_3_m_grouped.apply(lambda x: qs.stats.volatility(x["Close"])))
    data["percentage_1_y_volatility"] = data["asset_name"].map(data_1_y_grouped.apply(lambda x: qs.stats.volatility(x["Close"])))


    #data["cagr_3m"] = data["asset_name"].map(data_3m_cagr)
    #data["cagr_1y"] = data["asset_name"].map(data_1y_cagr)
    #data["cagr_3y"] = data["asset_name"].map(data_3y_cagr)

    return data


In [14]:
combined_df

,Date,Open,High,Low,Close,Adj Close,Volume,asset_name,ratio_sharpe,ratio_sortino,ratio_win_loss,ratio_drawdown,percentage_1_d_cagr,percentage_1_m_cagr,percentage_3_m_cagr,percentage_1_y_cagr,percentage_1_d_volatility,percentage_1_m_volatility,percentage_3_m_volatility,percentage_1_y_volatility
0,2007-05-30,91.580002,91.599998,91.500000,91.599998,78.043671,1550,BIL,0.021878,0.025966,0.652478,-0.013625,0.000000,0.005698,0.012473,-0.000150,0.00000,0.013144,0.011392,0.014439
1,2007-05-31,91.580002,91.599998,91.580002,91.599998,78.043671,11200,BIL,0.021878,0.025966,0.652478,-0.013625,0.000000,0.005698,0.012473,-0.000150,0.00000,0.013144,0.011392,0.014439
2,2007-06-01,91.620003,91.620003,91.620003,91.620003,78.060715,1450,BIL,0.021878,0.025966,0.652478,-0.013625,0.000000,0.005698,0.012473,-0.000150,0.00000,0.013144,0.011392,0.014439
3,2007-06-04,91.639999,91.639999,91.639999,91.639999,78.077782,1050,BIL,0.021878,0.025966,0.652478,-0.013625,0.000000,0.005698,0.012473,-0.000150,0.00000,0.013144,0.011392,0.014439
4,2007-06-05,91.660004,91.680000,91.639999,91.639999,78.077782,3750,BIL,0.021878,0.025966,0.652478,-0.013625,0.000000,0.005698,0.012473,-0.000150,0.00000,0.013144,0.011392,0.014439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66601,2024-03-22,85.690002,85.879997,84.510002,84.570000,84.570000,3923200,VNQ,0.243817,0.347044,0.941031,-0.757649,4.952548,0.182223,-0.088422,0.068837,0.07974,0.161601,0.170656,0.184994
66602,2024-03-25,84.650002,84.959999,84.050003,84.050003,84.050003,5262400,VNQ,0.243817,0.347044,0.941031,-0.757649,4.952548,0.182223,-0.088422,0.068837,0.07974,0.161601,0.170656,0.184994
66603,2024-03-26,84.269997,84.370003,83.690002,83.709999,83.709999,4923000,VNQ,0.243817,0.347044,0.941031,-0.757649,4.952548,0.182223,-0.088422,0.068837,0.07974,0.161601,0.170656,0.184994
66604,2024-03-27,84.669998,85.900002,84.470001,85.870003,85.870003,5191400,VNQ,0.243817,0.347044,0.941031,-0.757649,4.952548,0.182223,-0.088422,0.068837,0.07974,0.161601,0.170656,0.184994


In [31]:
percentage_allocations = {
    'SPY': 5,
    'MTUM': 10,
    'IWN': 5,
    'EFA': 10,
    'EEM': 10,
    'IEF': 5,
    'BWX': 5,
    'LQD': 5,
    'TLT': 5,
    'DBC': 10,
    'GLD': 10,
    'VNQ': 20
}

combined_df['percentage_allocation'] = combined_df['asset_name'].map(percentage_allocations) 

C:\Users\dhava\AppData\Local\Temp\ipykernel_31436\471682691.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['percentage_allocation'] = combined_df['asset_name'].map(percentage_allocations)


In [32]:
combined_df.dropna(subset=['percentage_allocation'], inplace=True)

C:\Users\dhava\AppData\Local\Temp\ipykernel_31436\970725549.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df.dropna(subset=['percentage_allocation'], inplace=True)


In [33]:
import pandas as pd
import numpy as np

# Assuming combined_df contains your combined DataFrame with asset_name, closing prices, and 'Date' column representing the trading dates

# Calculate 10-month moving average for each asset
combined_df['10_month_MA'] = combined_df.groupby('asset_name')['Close'].transform(lambda x: x.rolling(window=10).mean())

# Function to determine whether to go long or hold cash
def determine_action(row):
    if pd.isnull(row['10_month_MA']):  # If 10-month MA is not available, hold cash
        return 'Cash'
    elif row['Close'] > row['10_month_MA']:  # If close price is above 10-month MA, go long
        return 'Go Long'
    else:
        return 'Hold Cash'

# Apply the function to each row to determine the action
combined_df['Action'] = combined_df.apply(determine_action, axis=1)

# Filter only the last trading day of each month
last_day_of_month = combined_df.groupby([combined_df['Date'].dt.year, combined_df['Date'].dt.month])['Date'].max()

# Mark the action for the last trading day of each month as 'Hold Cash'
combined_df['LastDayOfMonth'] = combined_df['Date'].isin(last_day_of_month)
combined_df.loc[combined_df['LastDayOfMonth'], 'Action'] = 'Hold Cash'
combined_df.drop('LastDayOfMonth', axis=1, inplace=True)

# Drop unnecessary columns
combined_df.drop(['10_month_MA'], axis=1, inplace=True)

# Display the resulting DataFrame
print(combined_df)

C:\Users\dhava\AppData\Local\Temp\ipykernel_31436\499267383.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['10_month_MA'] = combined_df.groupby('asset_name')['Close'].transform(lambda x: x.rolling(window=10).mean())


            Date       Open       High        Low      Close  Adj Close  \
4257  2007-11-07  28.094999  28.094999  27.000000  27.000000  21.673990   
4258  2007-11-08  27.049999  27.129999  27.045000  27.110001  21.762299   
4259  2007-11-09  27.235001  27.235001  27.100000  27.200001  21.834543   
4260  2007-11-12  28.014999  28.014999  27.155001  27.225000  21.854601   
4261  2007-11-13  27.075001  27.075001  26.965000  27.014999  21.686028   
...          ...        ...        ...        ...        ...        ...   
66589 2024-03-06  86.879997  87.080002  86.239998  86.750000  86.750000   
66590 2024-03-07  87.209999  87.419998  86.300003  86.730003  86.730003   
66591 2024-03-08  87.500000  88.029999  87.300003  87.730003  87.730003   
66592 2024-03-11  87.540001  88.070000  86.940002  87.309998  87.309998   
66593 2024-03-12  87.150002  87.480003  86.370003  87.120003  87.120003   

        Volume asset_name  ratio_sharpe  ratio_sortino  ...  \
4257    277600        BWX     -0.061

C:\Users\dhava\AppData\Local\Temp\ipykernel_31436\499267383.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['Action'] = combined_df.apply(determine_action, axis=1)
C:\Users\dhava\AppData\Local\Temp\ipykernel_31436\499267383.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['LastDayOfMonth'] = combined_df['Date'].isin(last_day_of_month)
C:\Users\dhava\AppData\Local\Temp\ipykernel_31436\499267383.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

In [34]:
combined_df = combined_df[(combined_df['Action'] != 'Hold Cash') & (combined_df['Action'] != 'Cash')]

In [35]:
combined_df

,Date,Open,High,Low,Close,Adj Close,Volume,asset_name,ratio_sharpe,ratio_sortino,...,percentage_1_d_cagr,percentage_1_m_cagr,percentage_3_m_cagr,percentage_1_y_cagr,percentage_1_d_volatility,percentage_1_m_volatility,percentage_3_m_volatility,percentage_1_y_volatility,Action,percentage_allocation
4266,2007-11-20,27.275000,27.450001,27.225000,27.299999,21.914814,174800,BWX,-0.061105,-0.085152,...,-0.432164,0.011799,-0.109376,-0.026365,0.02518,0.052630,0.074750,0.091849,Go Long,5.0
4267,2007-11-21,27.500000,27.500000,27.350000,27.445000,22.031210,278400,BWX,-0.061105,-0.085152,...,-0.432164,0.011799,-0.109376,-0.026365,0.02518,0.052630,0.074750,0.091849,Go Long,5.0
4268,2007-11-23,27.420000,27.424999,27.400000,27.415001,22.007130,73800,BWX,-0.061105,-0.085152,...,-0.432164,0.011799,-0.109376,-0.026365,0.02518,0.052630,0.074750,0.091849,Go Long,5.0
4269,2007-11-26,28.235001,28.235001,27.379999,27.559999,22.123528,202600,BWX,-0.061105,-0.085152,...,-0.432164,0.011799,-0.109376,-0.026365,0.02518,0.052630,0.074750,0.091849,Go Long,5.0
4270,2007-11-27,27.750000,27.750000,27.375000,27.400000,21.995083,239600,BWX,-0.061105,-0.085152,...,-0.432164,0.011799,-0.109376,-0.026365,0.02518,0.052630,0.074750,0.091849,Go Long,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66541,2023-12-26,87.680000,88.599998,87.589996,88.360001,88.360001,4336600,VNQ,0.243817,0.347044,...,4.952548,0.182223,-0.088422,0.068837,0.07974,0.161601,0.170656,0.184994,Go Long,20.0
66545,2024-01-02,88.050003,89.260002,87.680000,89.120003,89.120003,6057900,VNQ,0.243817,0.347044,...,4.952548,0.182223,-0.088422,0.068837,0.07974,0.161601,0.170656,0.184994,Go Long,20.0
66591,2024-03-08,87.500000,88.029999,87.300003,87.730003,87.730003,4546400,VNQ,0.243817,0.347044,...,4.952548,0.182223,-0.088422,0.068837,0.07974,0.161601,0.170656,0.184994,Go Long,20.0
66592,2024-03-11,87.540001,88.070000,86.940002,87.309998,87.309998,3467200,VNQ,0.243817,0.347044,...,4.952548,0.182223,-0.088422,0.068837,0.07974,0.161601,0.170656,0.184994,Go Long,20.0


In [36]:
combined_df.drop('Action', axis = 1)

,Date,Open,High,Low,Close,Adj Close,Volume,asset_name,ratio_sharpe,ratio_sortino,...,ratio_drawdown,percentage_1_d_cagr,percentage_1_m_cagr,percentage_3_m_cagr,percentage_1_y_cagr,percentage_1_d_volatility,percentage_1_m_volatility,percentage_3_m_volatility,percentage_1_y_volatility,percentage_allocation
4266,2007-11-20,27.275000,27.450001,27.225000,27.299999,21.914814,174800,BWX,-0.061105,-0.085152,...,-0.359510,-0.432164,0.011799,-0.109376,-0.026365,0.02518,0.052630,0.074750,0.091849,5.0
4267,2007-11-21,27.500000,27.500000,27.350000,27.445000,22.031210,278400,BWX,-0.061105,-0.085152,...,-0.359510,-0.432164,0.011799,-0.109376,-0.026365,0.02518,0.052630,0.074750,0.091849,5.0
4268,2007-11-23,27.420000,27.424999,27.400000,27.415001,22.007130,73800,BWX,-0.061105,-0.085152,...,-0.359510,-0.432164,0.011799,-0.109376,-0.026365,0.02518,0.052630,0.074750,0.091849,5.0
4269,2007-11-26,28.235001,28.235001,27.379999,27.559999,22.123528,202600,BWX,-0.061105,-0.085152,...,-0.359510,-0.432164,0.011799,-0.109376,-0.026365,0.02518,0.052630,0.074750,0.091849,5.0
4270,2007-11-27,27.750000,27.750000,27.375000,27.400000,21.995083,239600,BWX,-0.061105,-0.085152,...,-0.359510,-0.432164,0.011799,-0.109376,-0.026365,0.02518,0.052630,0.074750,0.091849,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66541,2023-12-26,87.680000,88.599998,87.589996,88.360001,88.360001,4336600,VNQ,0.243817,0.347044,...,-0.757649,4.952548,0.182223,-0.088422,0.068837,0.07974,0.161601,0.170656,0.184994,20.0
66545,2024-01-02,88.050003,89.260002,87.680000,89.120003,89.120003,6057900,VNQ,0.243817,0.347044,...,-0.757649,4.952548,0.182223,-0.088422,0.068837,0.07974,0.161601,0.170656,0.184994,20.0
66591,2024-03-08,87.500000,88.029999,87.300003,87.730003,87.730003,4546400,VNQ,0.243817,0.347044,...,-0.757649,4.952548,0.182223,-0.088422,0.068837,0.07974,0.161601,0.170656,0.184994,20.0
66592,2024-03-11,87.540001,88.070000,86.940002,87.309998,87.309998,3467200,VNQ,0.243817,0.347044,...,-0.757649,4.952548,0.182223,-0.088422,0.068837,0.07974,0.161601,0.170656,0.184994,20.0


In [37]:
def map_asset_category(asset_name):
    if asset_name in ['SPY', 'MTUM', 'IWN', 'EFA', 'EEM', 'XHB', 'XLB', 'XLE', 'XLY', 'XLK', 'XLV', 'XLI', 'XLU', 'XLP', 'XLF', 'XLC', 'XLRE']:
        return "ETF"
    elif asset_name in ['DBC', 'GLD']:
        return "Gold"
    elif asset_name in ['BIL', 'IEF', 'BWX', 'LQD', 'TLT']:
        return "Treasury"
    elif asset_name == 'VNQ':
        return "REIT"
    else:
        return "Stock"

# Apply the function to create the 'asset_category' column
combined_df['asset_category'] = combined_df['asset_name'].apply(map_asset_category)

C:\Users\dhava\AppData\Local\Temp\ipykernel_31436\2488293010.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['asset_category'] = combined_df['asset_name'].apply(map_asset_category)


In [38]:
combined_df

,Date,Open,High,Low,Close,Adj Close,Volume,asset_name,ratio_sharpe,ratio_sortino,...,percentage_1_m_cagr,percentage_3_m_cagr,percentage_1_y_cagr,percentage_1_d_volatility,percentage_1_m_volatility,percentage_3_m_volatility,percentage_1_y_volatility,Action,percentage_allocation,asset_category
4266,2007-11-20,27.275000,27.450001,27.225000,27.299999,21.914814,174800,BWX,-0.061105,-0.085152,...,0.011799,-0.109376,-0.026365,0.02518,0.052630,0.074750,0.091849,Go Long,5.0,Treasury
4267,2007-11-21,27.500000,27.500000,27.350000,27.445000,22.031210,278400,BWX,-0.061105,-0.085152,...,0.011799,-0.109376,-0.026365,0.02518,0.052630,0.074750,0.091849,Go Long,5.0,Treasury
4268,2007-11-23,27.420000,27.424999,27.400000,27.415001,22.007130,73800,BWX,-0.061105,-0.085152,...,0.011799,-0.109376,-0.026365,0.02518,0.052630,0.074750,0.091849,Go Long,5.0,Treasury
4269,2007-11-26,28.235001,28.235001,27.379999,27.559999,22.123528,202600,BWX,-0.061105,-0.085152,...,0.011799,-0.109376,-0.026365,0.02518,0.052630,0.074750,0.091849,Go Long,5.0,Treasury
4270,2007-11-27,27.750000,27.750000,27.375000,27.400000,21.995083,239600,BWX,-0.061105,-0.085152,...,0.011799,-0.109376,-0.026365,0.02518,0.052630,0.074750,0.091849,Go Long,5.0,Treasury
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66541,2023-12-26,87.680000,88.599998,87.589996,88.360001,88.360001,4336600,VNQ,0.243817,0.347044,...,0.182223,-0.088422,0.068837,0.07974,0.161601,0.170656,0.184994,Go Long,20.0,REIT
66545,2024-01-02,88.050003,89.260002,87.680000,89.120003,89.120003,6057900,VNQ,0.243817,0.347044,...,0.182223,-0.088422,0.068837,0.07974,0.161601,0.170656,0.184994,Go Long,20.0,REIT
66591,2024-03-08,87.500000,88.029999,87.300003,87.730003,87.730003,4546400,VNQ,0.243817,0.347044,...,0.182223,-0.088422,0.068837,0.07974,0.161601,0.170656,0.184994,Go Long,20.0,REIT
66592,2024-03-11,87.540001,88.070000,86.940002,87.309998,87.309998,3467200,VNQ,0.243817,0.347044,...,0.182223,-0.088422,0.068837,0.07974,0.161601,0.170656,0.184994,Go Long,20.0,REIT
